In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib as mpl
from scipy.stats import entropy

mpl.rcParams['agg.path.chunksize'] = 10000
mpl.rcParams.update({'agg.path.chunksize': 10000})
mpl.rcParams["font.family"] = "Times New Roman"

import matplotlib.pyplot as plt

from matplotlib.ticker import FormatStrFormatter
import scipy.stats as ss
import scipy.stats as stats
import scipy.special as special
import matplotlib.animation as animation
pd.options.display.max_columns = None
from sklearn.metrics import roc_curve, roc_auc_score, auc

Set LDPC codeword length and get number of packets received.

In [2]:
# Set codeword length #
n = 540

if n == 54:
    N = 231880
if n == 88:
    N = 142290
if n == 124:
    N = 100980
if n == 540:
    N = 23188
if n == 880:
    N = 14229
if n == 1240:
    N = 10098

print("Codeword length: {}".format(n))
print("Number of codewords: {}".format(N))

Codeword length: 540
Number of codewords: 23188


Import the simulation data from csv files.

In [3]:
df_bit_level = pd.read_csv('bit-level-noise-estimation-540-440.csv', header=None)
print("Results data: {}".format(df_bit_level.shape))

Results data: (12521520, 10)


Create column names.

In [4]:
df_bit_level.columns = ['shiftedTrueBits',
                        'truePrec',
                        'recBits',
                        'model1DecodedBits',
                        'model1PrecMean',
                        'model1PrecVar',
                        'model2DecodedBits',
                        'model2PrecMean',
                        'model2PrecVar',
                        'modelIdealDecodedBits']

Calculate bit errors.

In [5]:
df_bit_level['trueBits'] = np.where(df_bit_level['shiftedTrueBits'] == -1, 0, 1)

df_bit_level['model1Error'] = np.where((df_bit_level['model1DecodedBits'] > 0.5) == df_bit_level['trueBits'], 0, 1)
df_bit_level['model2Error'] = np.where((df_bit_level['model2DecodedBits'] > 0.5) == df_bit_level['trueBits'], 0, 1)
df_bit_level['noFecError'] = np.where((df_bit_level['recBits'] > 0) == df_bit_level['trueBits'], 0, 1)
df_bit_level['modelIdealError'] = np.where((df_bit_level['modelIdealDecodedBits'] > 0.5) == df_bit_level['trueBits'], 0, 1)


Calculate model evidence per bit: 
- $p(\theta_{m}|r)_{t} = \sum_{i} [b_t=i] \mathcal{N}_{i}(r_t|\mu_{i};\gamma_{t}^{-1})$
- $\ell(\theta_{m}|r)_{t} = 10 \log_{10}(\frac{p(\theta_{m=1}|r)_{t}}{p(\theta_{m=2}|r)_{t}}) $ in dB.

In [6]:
df_bit_level['model1Likehood'] = (1 - df_bit_level['model1DecodedBits']) * stats.norm.pdf(df_bit_level['recBits'], loc=-1, scale=np.sqrt(1 / df_bit_level['model1PrecMean'])) + \
(df_bit_level['model1DecodedBits']) * stats.norm.pdf(df_bit_level['recBits'], loc=1, scale=np.sqrt(1 / df_bit_level['model1PrecMean']))

df_bit_level['model2Likehood'] = (1 - df_bit_level['model2DecodedBits']) * stats.norm.pdf(df_bit_level['recBits'], loc=-1, scale=np.sqrt(1 / df_bit_level['model2PrecMean'])) + \
(df_bit_level['model2DecodedBits']) * stats.norm.pdf(df_bit_level['recBits'], loc=1, scale=np.sqrt(1 / df_bit_level['model2PrecMean']))

df_bit_level['likehoodRatio_db'] = 10*np.log10(df_bit_level['model1Likehood'] / df_bit_level['model2Likehood'])

np.log(df_bit_level['model1Likehood']).mean(), np.log(df_bit_level['model2Likehood']).mean()

(-1.2186763809094139, -1.1449882934498379)

Create packet identification.

In [7]:
ids = [[i]*n for i in range(N)]
packet_ids = [item for sublist in ids for item in sublist]
df_bit_level['packet_id'] = packet_ids

Create cumulative bit error columns.

In [8]:
df_bit_level['model1ErrorsCum'] = df_bit_level['model1Error'].cumsum()
df_bit_level['model2ErrorsCum'] = df_bit_level['model2Error'].cumsum()
df_bit_level['noFecErrorsCum'] = df_bit_level['noFecError'].cumsum()
df_bit_level['modelIdealErrorsCum'] = df_bit_level['modelIdealError'].cumsum()

In [9]:
df_bit_level[['model1ErrorsCum', 'model2ErrorsCum', 'modelIdealErrorsCum']].iloc[-1]

model1ErrorsCum        1561297
model2ErrorsCum        1555024
modelIdealErrorsCum    1554201
Name: 12521519, dtype: int64

Create SNR estimation and BER plots.

In [19]:
fig, ax = plt.subplots(4, gridspec_kw={'height_ratios':[6.8, 6.8, 5.5, 5.5]}, sharex=True)
fig.set_size_inches(8.5, 12.5)

y = df_bit_level['model1PrecMean']
error = np.sqrt(df_bit_level['model1PrecVar'])

ax[0].plot(10*np.log10(y - error), alpha=0.2, rasterized=True, color='purple', label='1 Standard deviation')
ax[0].plot(10*np.log10(y + error), alpha=0.2, rasterized=True, color='purple')

ax[0].plot(10*np.log10(df_bit_level['model1PrecMean']), label='Estimated SNR', alpha=0.78, linewidth=1)
ax[0].plot(10*np.log10(df_bit_level['truePrec']), rasterized=True, label='True SNR', color='black', alpha=0.78, linewidth=0.8)

# ax[0].fill_between(range(10000*n), 10*np.log10(y - error), 10*np.log10(y + error), alpha=0.2, rasterized=True, color='purple', label='1 Standard deviation')

ax[0].set_ylabel("Signal to noise ratio (dB)", color="black", fontsize=14)
# ax[0].set_xlabel("Packets received", color="black", fontsize=12)
ax[0].set_title("(a) Model I results", color="black", fontsize=14)

ax2 = ax[0].twinx()

model1cumerrors = df_bit_level[['packet_id', 'model1Error']].groupby('packet_id').sum().cumsum()
model1cumerrors = model1cumerrors['model1Error'] / ((model1cumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), model1cumerrors,
         label='SNR estimator (BER)',
         color='red',
         linewidth=1.2)
# ax2.plot(df[25] / 44, label='SNR Estimator (BER)', color='red', linewidth=1, alpha=0.1)
noFECcumerrors = df_bit_level[['packet_id', 'noFecError']].groupby('packet_id').sum().cumsum()
noFECcumerrors = noFECcumerrors['noFecError'] / ((noFECcumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), noFECcumerrors, 
         label='Uncoded (BER)',
         color='gray',
         rasterized=True,
         linewidth=1.5)
modelidealcumerrors = df_bit_level[['packet_id', 'modelIdealError']].groupby('packet_id').sum().cumsum()
modelidealcumerrors = modelidealcumerrors['modelIdealError'] / ((modelidealcumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), modelidealcumerrors,
         label='Optimum (BER)',
         color='black', 
         linestyle='--',
         rasterized=True,
         linewidth=1)
ax2.set_ylabel("Cumulative bit error rate", color="black", fontsize=14)
# ax2.set_yscale('symlog', linthreshy=0.0015)
ax2.set_yscale('log')
ax2.set_yticks([(10)**-3, (10)**-2, (10)**-1, (10)**0])
ax[0].set_xlim([0, N*n])
ax[0].set_xticks(np.arange(0, (N+1)*n, n*10000))
ax[0].set_xticklabels(np.arange(0, (N+1), 10000))

ax[0].grid()

handles, labels = ax[0].get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(handles + handles2, labels + labels2, loc='lower right', facecolor='white', framealpha=0.9)

y = df_bit_level['model2PrecMean']
error = np.sqrt(df_bit_level['model2PrecVar'])

ax[1].plot(10*np.log10(y - error), alpha=0.8, rasterized=True, color='#33ff33', label='1 Standard deviation')
ax[1].plot(10*np.log10(y + error), alpha=0.8, rasterized=True, color='#33ff33')

# ax[1].fill_between(range(10000*n), 10*np.log10(y - error), 10*np.log10(y + error), alpha=0.8, rasterized=True, color='#33ff33', label='1 Standard deviation')

ax[1].plot(10*np.log10(df_bit_level['model2PrecMean']), label='Estimated SNR', color='#ff9900', alpha=0.8, linewidth=1)
ax[1].plot(10*np.log10(df_bit_level['truePrec']), rasterized=True, label='True SNR', alpha=0.78, color='black', linewidth=0.8)

ax[1].set_ylabel("Signal to noise ratio (dB)", color="black", fontsize=14)
# # ax[1].set_xlabel("Packets received", color="black", fontsize=12)
# # ax[1].set_ylim([-25, 25])
ax[1].set_title("(b) Model II results", color="black", fontsize=14)

ax2 = ax[1].twinx()
model1cumerrors = df_bit_level[['packet_id', 'model2Error']].groupby('packet_id').sum().cumsum()
model1cumerrors = model1cumerrors['model2Error'] / ((model1cumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), model1cumerrors,
         label='SNR estimator (BER)',
         color='red',
         linewidth=1.2)
# ax2.plot(df[25] / 44, label='SNR Estimator (BER)', color='red', linewidth=1, alpha=0.1)
noFECcumerrors = df_bit_level[['packet_id', 'noFecError']].groupby('packet_id').sum().cumsum()
noFECcumerrors = noFECcumerrors['noFecError'] / ((noFECcumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), noFECcumerrors, 
         label='Uncoded (BER)',
         color='gray',
         rasterized=True,
         linewidth=1.5)
modelidealcumerrors = df_bit_level[['packet_id', 'modelIdealError']].groupby('packet_id').sum().cumsum()
modelidealcumerrors = modelidealcumerrors['modelIdealError'] / ((modelidealcumerrors.index + 1) * n)

ax2.plot(np.arange(n, ((N+1)*n), n), modelidealcumerrors,
         label='Optimum (BER)',
         color='black', 
         linestyle='--',
         rasterized=True,
         linewidth=1)
ax2.set_ylabel("Cumulative bit error rate", color="black", fontsize=14)
# ax2.set_yscale('symlog', linthreshy=0.0015)
ax2.set_yscale('log')
ax2.set_yticks([(10)**-3, (10)**-2, (10)**-1, (10)**0])
ax[1].set_xlim([0, N*n])
ax[1].set_xticks(np.arange(0, (N+1)*n, n*10000))
ax[1].set_xticklabels(np.arange(0, (N+1), 10000))
# ax2.set_ylim([0.001, 0.3])
# ax2.set_xlim()
# ax2.semilogy(range(10000), np.log(range(10000)))
ax[1].grid()

handles, labels = ax[1].get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(handles + handles2, labels + labels2, loc='lower right', facecolor='white', framealpha=0.9)

# ax3 = ax[2].twinx()
# df_error_correction[['model1errors', 'model2errors']].plot(kind='bar', ax=ax3, stacked=True)
ax[2].scatter(np.arange(n, ((N+1)*n), n), df_bit_level[['packet_id', 'model1Error']].groupby('packet_id').sum(),
              color='blue',
              marker='.',
              label='Model I',
              alpha=0.8)
ax[2].scatter(np.arange(n, ((N+1)*n), n), df_bit_level[['packet_id', 'model2Error']].groupby('packet_id').sum(),
              color='#ff9900',
              marker='x',
              label='Model II',
              alpha=0.8)

ax[2].set_ylabel("Bit errors per packet", color="black", fontsize=14)
ax[2].set_title("(c) Bit error comparison", color="black", fontsize=14)
ax[2].grid()
handles, labels = ax[2].get_legend_handles_labels()
# handles2, labels2 = ax2.get_legend_handles_labels()
ax[2].legend(handles, labels, loc='upper right', facecolor='white', framealpha=0.8)


ax[3].set_ylabel("Ratio of model evidence (dB)", color="black", fontsize=14)
ax[3].plot(df_bit_level['likehoodRatio_db'], color='black', alpha=0.5)
ax[3].set_xlabel("Packets received", color="black", fontsize=14)
ax[3].set_title("(d) Model evidence comparison", color="black", fontsize=14)
ax[3].grid()
# ax[3].vlines(np.arange(0, 12521520, 540), -50, 50, linestyle=':', alpha=0.5)
# plt.xticks(np.arange(12320000, 12521520, 88))

# ax3 = ax[3].twinx()
# ax3.plot(np.arange(1000, ((N)*n)+1000, 1000),
#          df_bit_level[['likehoodRatio_db']].groupby(df_bit_level.index // 1000).mean().values,
#          color='red')

fig.tight_layout(pad=1.5)

plt.savefig('non-stationary-result.pdf', bbox_inches='tight')

plt.show()

TotalBERModel1 = df_bit_level['model1ErrorsCum'].iloc[-1] / df_bit_level.shape[0]
TotalBERModel2 = df_bit_level['model2ErrorsCum'].iloc[-1] / df_bit_level.shape[0]
print("Model 1 BER: {}".format(TotalBERModel1))
print("Model 2 BER: {}".format(TotalBERModel2))

In [3]:
# df_bit_level['diff'] = df_bit_level['model1Error'] - df_bit_level['model2Error']

In [196]:
# plt.hist(df_bit_level['model1Likehood'], 1000)
# plt.hist(df_bit_level['model2Likehood'], 1000, alpha=0.6)
# plt.show()

In [4]:
# model1err = df_bit_level[df_bit_level['diff'] == 1]['bitsTruePrec'].values
# model2err = df_bit_level[df_bit_level['diff'] == -1]['bitsTruePrec'].values

In [89]:
# plt.boxplot([model1err, model2err])
# plt.xticks([1, 2], ['model1err', 'model2err'])
# plt.show()

In [90]:
# fig, axs = plt.subplots(1)
# p = axs.scatter(df_bit_level['llhr_db'],
#                 df_bit_level['diff'], 
#                 marker='.',
#                 c=df_bit_level['bitsTruePrec'],
#                 cmap='inferno')
# # q = axs[1].scatter(df_bit_level['llhr_db'],
# #             df_bit_level['model2errors'], 
# #             c=df_bit_level['snr'], 
# #             marker='.',
# #             alpha=0.7,
# #             cmap='magma')

# # fig.colorbar(p, ax=axs[0])
# fig.colorbar(p, ax=axs)
# # axs[1].colorbar()
# plt.grid()
# plt.show()

In [91]:
# # (df_bit_level['model1errorsCum'] / df_bit_level.index).plot(label='M1')
# # (df_bit_level['model2errorsCum'] / df_bit_level.index).plot(label='M2')
# # (df_bit_level['noFECerrorsCum'] / df_bit_level.index).plot(label='NoFEC')
# # (df_bit_level['modelidealerrorsCum'] / df_bit_level.index).plot(label='ideal', linestyle='--')
# plt.scatter(range(df_bit_level.shape[0]), df_bit_level['model1error'] - df_bit_level['model2error'])
# plt.plot(10*np.log10(df_bit_level['bitsTruePrec']), label='TrueSNR', color='black')
# # plt.plot(10*np.log10(df_bit_level['model1PrecMean']), label='M1SNR', color='blue', alpha=0.5)
# # plt.plot(10*np.log10(df_bit_level['model2PrecMean']), label='M2SNR', color='orange', alpha=0.5)
# # plt.yscale('log')
# # plt.legend()
# plt.show()

In [92]:
# plt.scatter(np.arange(n, ((N+1)*n), n), df_bit_level[['packet_id', 'model1error']].groupby('packet_id').sum())
# plt.scatter(np.arange(n, ((N+1)*n), n), df_bit_level[['packet_id', 'model2error']].groupby('packet_id').sum())
# plt.plot(10*np.log10(df_bit_level['bitsTruePrec']), label='True SNR', color='black', alpha=0.78, linewidth=0.8)

# plt.show()

In [5]:
# 1/2, 2/3, 3/4, 5/6, 10/11, 20/21, 50/51, 400/401

In [6]:
# plt.hist(df_bit_level['model1PrecMean'] - df_bit_level['bitsTruePrec'], 500)
# plt.hist(df_bit_level['model2PrecMean'] - df_bit_level['bitsTruePrec'], 500)

# plt.show()


In [80]:
# plt.scatter(df_bit_level['model1entropy'], df_bit_level['model2entropy'])
# plt.plot(df_bit_level['model2entropy'])
# plt.plot(df_bit_level['model1PrecMean'])
# plt.plot(df_bit_level['bitsTruePrec'], color='black')
# plt.show()

In [81]:
# fpr, tpr, thresholds = roc_curve(df_bit_level['bits'], df_bit_level['model1decodedbits'])
# roc_auc = roc_auc_score(df_bit_level['bits'], df_bit_level['model1decodedbits'])

# fpr2, tpr2, thresholds2 = roc_curve(df_bit_level['bits'], df_bit_level['model2decodedbits'])
# roc_auc2 = roc_auc_score(df_bit_level['bits'], df_bit_level['model2decodedbits'])

# # Plot ROC curve
# plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
# plt.plot(fpr2, tpr2, label='ROC curve (area = %0.3f)' % roc_auc2)
# plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate or (1 - Specifity)')
# plt.ylabel('True Positive Rate or (Sensitivity)')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

In [7]:
# 140000*88, 142290*88

In [36]:
# df_bit_level.iloc[7920000:8008000, :].shape

In [8]:
# df_bit_level_temp = df_bit_level.iloc[12320000:12521520, :].copy()

In [25]:
# # n=88
# # plt.scatter(np.arange(0, 880000), df_bit_level['recbits'], color='black', label='Received bit values', alpha=0.3, marker='.')

# # plt.scatter(df_bit_level[df_bit_level['model1correct'] == 1]['recbits'].index.values, df_bit_level[df_bit_level['model1correct'] == 1]['recbits'], color='green', label='Received bit values', alpha=0.3, marker='.')
# # plt.scatter(df_bit_level[df_bit_level['model1correct'] == 0]['recbits'].index.values, df_bit_level[df_bit_level['model1correct'] == 0]['recbits'], color='red', label='Received bit values', alpha=0.3, marker='.')

# y = df_bit_level_temp['model1PrecMean']
# error = np.sqrt(df_bit_level_temp['model1PrecVar'])

# plt.plot(10*np.log10(df_bit_level_temp['bitsTruePrec']),
#          rasterized=True,
#          label='True SNR',
#          color='black',
#          alpha=0.78,
#          linewidth=1)
# plt.plot(10*np.log10(df_bit_level_temp['model1PrecMean']),
#          rasterized=True,
#          label='Model I estimated SNR',
#          alpha=0.78,
#          linewidth=1)

# plt.fill_between(range(12320000,12521520), 10*np.log10(y - error),
#                  10*np.log10(y + error),
#                  alpha=0.2,
#                  rasterized=True,
#                  color='purple',
#                  label='1 Standard deviation')

# y = df_bit_level_temp['model2PrecMean']
# error = np.sqrt(df_bit_level_temp['model2PrecVar'])

# plt.fill_between(range(12320000,12521520),
#                  10*np.log10(y - error),
#                  10*np.log10(y + error),
#                  alpha=0.8,
#                  rasterized=True,
#                  color='#33ff33',
#                  label='1 Standard deviation')

# plt.plot(10*np.log10(df_bit_level_temp['model2PrecMean']),
#          label='Model II estimated SNR',
#          color='#ff9900',
#          alpha=0.8,
#          linewidth=1)

# plt.scatter(np.arange(12320000, 12521520, 88),
#             df_bit_level_temp[['packet_id', 'model1error']].groupby('packet_id').sum(),
#             color='blue',
#             marker='.',
#             label='Model I',
#             alpha=0.9)
# plt.scatter(np.arange(12320000, 12521520, 88),
#             df_bit_level_temp[['packet_id', 'model2error']].groupby('packet_id').sum(),
#             color='#ff9900',
#             marker='x',
#             label='Model II',
#             alpha=0.9)
# plt.vlines(np.arange(12320000, 12521520, 88), -15, 50, linestyle=':', alpha=0.5)
# plt.xticks(np.arange(12320000, 12521520, 88))
# plt.legend()
# plt.xlim([12420250, 12422250])
# plt.ylim([-6, 30])
# plt.ylabel("Signal to noise ratio (dB) and Bit errors", color="black", fontsize=14)
# plt.xlabel("Bits received", color="black", fontsize=14)
# plt.title("Model comparison", color="black", fontsize=14)
# plt.grid()
# plt.ticklabel_format(useOffset=False, style='plain')
# plt.savefig('model-comparison-result.pdf', bbox_inches='tight')
# plt.show()

In [ ]:
# df_error_correction['cumulative_bits'] = [(i+1)*44 for i in range(N)]
# df_error_correction['cumulative_bits'] = [(i+1)*440 for i in range(N)]

In [9]:
# l = [[i]*n for i in range(N)]
# flat_list = [item for sublist in l for item in sublist]
# len(flat_list)

In [95]:
# df_bit_level['packet_number'] = flat_list

In [96]:
# df_error_correction.shape

In [10]:
# m1 = df_bit_level[['packet_number', 'model1lh']].groupby('packet_number').mean()
# m1errors = df_bit_level[['packet_number', 'model1error']].groupby('packet_number').sum()
# m2errors = df_bit_level[['packet_number', 'model2error']].groupby('packet_number').sum()
# m2 = df_bit_level[['packet_number', 'model2lh']].groupby('packet_number').mean()
# llh = df_bit_level[['packet_number', 'llhr_db']].groupby('packet_number').mean()
# snr = df_bit_level[['packet_number', 'bitsTruePrec']].groupby('packet_number').mean()
# # errdiv = df_error_correction['model2errors'] - df_error_correction['model1errors']

In [11]:
# plt.scatter(np.log(m2),
#             m2errors,  
#             marker='.',
#             c=snr.values)
# # plt.scatter(df_error_correction['m2'], df_error_correction['model2errors'], color='orange', marker='x')
# plt.colorbar()
# plt.show()

In [462]:
# plt.scatter(10*np.log10(df_bit_level[0]), 10*np.log10(df_bit_level[1]))
# plt.plot(range(75), range(75), linestyle='--', color='black')
# plt.xlabel("True SNR")
# plt.ylabel("Estimated SNR")
# plt.grid()
# plt.show()

In [37]:
# mape = np.mean(np.abs((df_bit_level[0] - df_bit_level[1]) / df_bit_level[0])) * 100
# TotalBER = (df_error_correction[26] / df_error_correction['cumulative_bits'])[9999]
# print("MAPE: {}%".format(mape))
# print("BER: {}".format(TotalBER))

In [30]:
# df[22] df[23] df[25] df[27] df[29]

In [38]:
# fig, ax = plt.subplots()
# fig.set_size_inches(9.5, 4.5)
# ax.plot(10*np.log10(df[22]), label='True SNR', color='black')
# ax.plot(10*np.log10(df[23]), label='Estimated SNR', alpha=0.78, linewidth=1)

# y = df[23]
# error = np.sqrt(df[24])
# plt.fill_between(range(N), 10*np.log10(y - error), 10*np.log10(y + error), alpha=0.2, color='purple', label='1 Standard Deviation')
# ax.set_ylabel("SNR (dB)", color="black", fontsize=12, fontname="Arial")
# ax.set_xlabel("Packets received", color="black", fontsize=12, fontname="Arial")

# ax2 = ax.twinx()
# ax2.plot(df[26] / df['cumulative_bits'], label='SNR Estimator (BER)', color='red', linewidth=1)
# # ax2.plot(df[25] / 44, label='SNR Estimator (BER)', color='red', linewidth=1, alpha=0.1)
# ax2.plot(df[28] / df['cumulative_bits'], label='Uncoded (BER)', color='gray', linewidth=0.9)
# ax2.plot(df[30] / df['cumulative_bits'], label='Optimum (BER)', color='black', linestyle=':')
# ax2.set_ylabel("Cumulative Bit error rate", color="black", fontsize=12, fontname="Arial")
# # ax2.set_yscale('symlog', linthreshy=0.0015)
# ax2.set_yscale('log')
# ax2.set_yticks([(10)**-3, (10)**-2, (10)**-1, (10)**0])
# ax.set_xlim([0, N])
# # ax2.set_ylim([0.001, 0.3])
# # ax2.set_xlim()
# # ax2.semilogy(range(10000), np.log(range(10000)))
# ax.grid()

# handles, labels = ax.get_legend_handles_labels()
# handles2, labels2 = ax2.get_legend_handles_labels()
# ax.legend(handles + handles2, labels + labels2, loc='lower right', facecolor='white', framealpha=0.8)
# plt.savefig('stationary-result.pdf')
# plt.show()

In [14]:
# pp = ProgressPlot(plot_names=["SNR", "BER"], 
#                   line_names=["True_SNR", "Estimated_SNR", "Optimal_BER", "Actual_BER"],
#                   y_lim=[[-15, 14],[0, 0.2]])

# for i in range(10000):
#     y_update_dict = {'SNR': {'True_SNR': 10*np.log10(df.loc[i, 22]),
#                              'Estimated_SNR': 10*np.log10(df.loc[i, 23])},
#                      'BER': {'Optimal_BER': df.loc[i, 30] / df.loc[i, 'cumulative_bits'],
#                              'Actual_BER': df.loc[i, 26] / df.loc[i, 'cumulative_bits']}}
#     pp.update(y_update_dict)
# pp.finalize()